In [9]:
from repartition import solve_assignment
import random
import math

In [6]:
random.seed(42)

students = [f"E{i}" for i in range(1, 24)]  # 25 élèves
all_projects = [f"P{j}" for j in range(1, 7)]  # 12 projets
scores = {}

# On simule des préférences : chaque élève tire 7 projets distincts et reçoit des notes 6..1
for e in students:
    prefs = random.sample(all_projects, 6)
    for rank, p in enumerate(prefs, start=1):
        scores[(e, p)] = 7 - rank  # 10 pour le 1er choix, 1 pour le 10e

print("\n=== Inputs ===")
print("students")
print(students)
print("all_projects")
print(all_projects)  
print("scores")
print(scores)


=== Inputs ===
students
['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23']
all_projects
['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
scores
{('E1', 'P6'): 6, ('E1', 'P1'): 5, ('E1', 'P5'): 4, ('E1', 'P3'): 3, ('E1', 'P2'): 2, ('E1', 'P4'): 1, ('E2', 'P2'): 6, ('E2', 'P6'): 5, ('E2', 'P1'): 4, ('E2', 'P3'): 3, ('E2', 'P4'): 2, ('E2', 'P5'): 1, ('E3', 'P1'): 6, ('E3', 'P6'): 5, ('E3', 'P5'): 4, ('E3', 'P4'): 3, ('E3', 'P3'): 2, ('E3', 'P2'): 1, ('E4', 'P5'): 6, ('E4', 'P2'): 5, ('E4', 'P4'): 4, ('E4', 'P1'): 3, ('E4', 'P6'): 2, ('E4', 'P3'): 1, ('E5', 'P1'): 6, ('E5', 'P2'): 5, ('E5', 'P4'): 4, ('E5', 'P5'): 3, ('E5', 'P3'): 2, ('E5', 'P6'): 1, ('E6', 'P2'): 6, ('E6', 'P3'): 5, ('E6', 'P1'): 4, ('E6', 'P4'): 3, ('E6', 'P6'): 2, ('E6', 'P5'): 1, ('E7', 'P3'): 6, ('E7', 'P6'): 5, ('E7', 'P5'): 4, ('E7', 'P1'): 3, ('E7', 'P2'): 2, ('E7', 'P4'): 1, ('E8', 'P4'): 6, ('E8', 'P1'): 5, ('E8', 'P3'): 4,

In [19]:
# CAS 2 (optionnel) : imposer K = ceil(N/4) pour forcer le maximum possible de groupes de 4
K_force = math.ceil(len(students) / 4)  # pour 25 → 7
selected2, assignment2, obj2 = solve_assignment(
    scores,
    students=students,
    projects=all_projects,
    min_size=3,
    max_size=4,
    k_projects=K_force,     # force exactement K groupes
    lexicographic=False,    # une seule phase (le score est l'objectif)
)
assignment2

{'P3': ['E18', 'E22', 'E23', 'E7'],
 'P4': ['E13', 'E16', 'E8'],
 'P6': ['E1', 'E11', 'E12', 'E9'],
 'P5': ['E17', 'E20', 'E21', 'E4'],
 'P1': ['E10', 'E15', 'E19', 'E3'],
 'P2': ['E14', 'E2', 'E5', 'E6']}

In [ ]:
print("\n=== Solution avec K fixé =", K_force, "(max de groupes de 4, règle dure) ===")
print("Projets sélectionnés :", sorted(selected2), f"(K={len(selected2)})")
for p in sorted(assignment2):
    print(f"{p}  (n={len(assignment2[p])})  ← {assignment2[p]}")
print("Score total :", obj2)

In [18]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# ===== Données =====
scores = scores
assignations = assignment

# ===== Construction du DataFrame pivoté (lignes E??, colonnes P??) =====
df = pd.Series(scores).unstack()  # colonnes = P??, index = E??
# Tri naturel E1..E23 et P1..P6
df = df.loc[sorted(df.index, key=lambda x: int(x[1:])),
            sorted(df.columns, key=lambda x: int(x[1:]))]

# ===== Colormap blanc -> vert foncé =====
# (blanc pour les plus faibles, vert foncé pour les plus forts)
cmap = LinearSegmentedColormap.from_list(
    "white_to_green",
    ["#ffffff", "#a5d6a7", "#66bb6a", "#2e7d32"]  # clair -> foncé
)

# ===== Fonction de style : encadrement des cellules assignées =====
def highlight_assigned(dataframe: pd.DataFrame) -> pd.DataFrame:
    styles = pd.DataFrame("", index=dataframe.index, columns=dataframe.columns)
    # Inverser la dict pour trouver, pour chaque étudiant, son projet assigné
    etu_to_proj = {}
    for p, etus in assignations.items():
        for e in etus:
            etu_to_proj[e] = p
    # Appliquer le bord noir épais
    for e, p in etu_to_proj.items():
        if e in styles.index and p in styles.columns:
            styles.loc[e, p] = "border: 3px solid black"
    return styles

# ===== Création du Styler =====
styler = (
    df.style
      .format("{:.0f}")                         
      .background_gradient(cmap=cmap, axis=None)
      .apply(highlight_assigned, axis=None)
      .set_properties(**{"text-align": "center"})
      .set_table_styles([
          {"selector": "th", "props": [("text-align", "center")]},
          {"selector": "td", "props": [("min-width", "3em"), ("height", "2.2em")]},
          {"selector": "table", "props": [("border-collapse", "collapse"),
                                          ("font-family", "sans-serif")]}
      ])
)


# ===== Affichage dans un notebook (Jupyter, VS Code, etc.) =====
# Dans un notebook, décommente la ligne suivante :
display(styler)

# ===== Export HTML autonome =====
#html_path = "tableau_scores_assignations.html"
#with open(html_path, "w", encoding="utf-8") as f:
#    f.write(styler.to_html())

#print(f"Fichier HTML généré : {html_path}")


,P1,P2,P3,P4,P5,P6
E1,5,2,3,1,4,6
E2,4,6,3,2,1,5
E3,6,1,2,3,4,5
E4,3,5,1,4,6,2
E5,6,5,2,4,3,1
E6,4,6,5,3,1,2
E7,3,2,6,1,4,5
E8,5,2,4,6,1,3
E9,5,1,3,2,4,6
E10,6,5,2,1,3,4
